In [4]:
import pickle
import pandas as pd
import sklearn
import numpy as np
import scipy
# load the dic file

from utils import *

path = 'cleaned/module_dict.pickle'
with open(path, 'rb') as f:
    module_dict = pickle.load(f)

In [5]:
tissues = ['islet', 'liver', 'adipose', 'kidney', 'gastroc']
signs = ['SIGNED', 'UNSIGNED']

X_train, X_test, Y_train, Y_test = get_data('islet','SIGNED','red')

In [6]:
# print the shape of training / testing
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((441, 764), (50, 764), (441, 4), (50, 4))

In [8]:
# use sklearn knn regression method to train a model then predict
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

# train a model
rds = np.random.RandomState(0)
model = KernelRidge() # no randomness, closed form solution
model.get_params()

{'alpha': 1,
 'coef0': 1,
 'degree': 3,
 'gamma': None,
 'kernel': 'linear',
 'kernel_params': None}

In [ ]:
param_grid = [{"kernel": ['poly'],
               "alpha": [0.1,100,1000],
               "gamma": [0.1,1,10],
               "degree": [2, 3, 4, 5]},
              {"kernel": ['linear', 'rbf', 'sigmoid', 'laplacian'],
               "alpha": [0.1,100,1000],
               "gamma": [0.1,1,10],}]
gsh = GridSearchCV(estimator=model, param_grid=param_grid)
gsh.fit(X_train, Y_train)
# predict
Y_pred = gsh.predict(X_test)

In [24]:
# visualize the gsh.cv_results_
rank = gsh.cv_results_['rank_test_score']
para = gsh.cv_results_['params']
# print para based on rank
for i in rank:
    print(para[int(i)-1])

{'alpha': 0.1, 'degree': 2, 'gamma': 0.1, 'kernel': 'poly'}
{'alpha': 0.1, 'degree': 3, 'gamma': 10, 'kernel': 'poly'}
{'alpha': 0.1, 'degree': 5, 'gamma': 10, 'kernel': 'poly'}
{'alpha': 0.1, 'degree': 2, 'gamma': 1, 'kernel': 'poly'}
{'alpha': 0.1, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
{'alpha': 1000, 'degree': 2, 'gamma': 0.1, 'kernel': 'poly'}
{'alpha': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'}
{'alpha': 100, 'degree': 2, 'gamma': 0.1, 'kernel': 'poly'}
{'alpha': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
{'alpha': 0.1, 'degree': 3, 'gamma': 1, 'kernel': 'poly'}
{'alpha': 100, 'degree': 5, 'gamma': 0.1, 'kernel': 'poly'}
{'alpha': 0.1, 'gamma': 1, 'kernel': 'laplacian'}
{'alpha': 100, 'degree': 4, 'gamma': 10, 'kernel': 'poly'}
{'alpha': 0.1, 'degree': 5, 'gamma': 1, 'kernel': 'poly'}
{'alpha': 100, 'degree': 2, 'gamma': 10, 'kernel': 'poly'}
{'alpha': 100, 'degree': 4, 'gamma': 1, 'kernel': 'poly'}
{'alpha': 100, 'degree': 2, 'gamma': 1, 'kernel': 'poly'}
{'alpha': 1000

In [25]:
# compute the average mean percentage error
def mean_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true),axis=0) * 100

per_err = mean_percentage_error(Y_test, Y_pred)
# compute the mean percentage error
per_err_mean = np.mean(per_err,axis=0)
# merge per_err and per_err_mean as pandas dataframe
per_err['mean'] = per_err_mean
per_err

glucose          16.245811
weight            6.577401
insulin         111.177375
triglyceride     34.115043
mean             42.028907
dtype: float64